In [1]:
import numpy as np
import numpy as np
from numpy import linalg as la
from sklearn import preprocessing as pre
import random
import math
from numpy import ma

In [ ]:
def get_label_matrix(X):

    '''
    Target 1: choose the front 20% data of each class as labeled sample
    Target 2: rearrange all the points and let the front l points be labeled
    '''
    
    return X,Yl,l

In [1]:
def classify(X,Yl,c,num_of_adaptive_neighbours=9):
    
    '''
    X=[X1,X2,...,Xv]  --list of feature matrixes
    Yl --labale matrix
    '''

    #Standardlize:
    n,_=X[0].shape
    v=len(X)#v:number of different views
    for i in range(v):
        X[i]=pre.scale(X[i],axis=1)#You need to import sklearn.preprocessing as pre before using the standardlization module.
    print("Preprocession finished...")
    
    #Initialize:
    lambda_=random.randint(1,30)
    l=Yl.shape[0]#l denotes the number of labeled points
    u=n-l#u denoted the number of unlabeled points
    W=np.ones((v),dtype=np.float32)/v#W records the weight of each view
    alpha=np.zeros((n),dtype=np.float32)
    S=np.zeros((n,n),dtype=np.float32)#S denotes the similarity matrix
    Dist=np.zeros((n,n),dtype=np.float32)#Dist denotes the distance matrix
    F=np.zeros((n,c),dtype=np.float32)#F denotes the indicator matrix
    F[:l]=Yl
    k_=num_of_adaptive_neighbours
    for i in range(n):
        for j in range(n):
            for k in range(v):
                Dist[i][j]+=W[k]*(la.norm(X[k][i,:]-X[k][j,:]))**2
        ascending_order_index=np.argsort(Dist[i])[1:k_+1]
        dist_of_neighbours_sorted_ascending=Dist[i][ascending_order_index]
        alpha[i]=0.5*(k_*dist_of_neighbours_sorted_ascending[k_-1]-np.sum(dist_of_neighbours_sorted_ascending))
        S[i][ascending_order_index]=(np.mean(dist_of_neighbours_sorted_ascending))/alpha[i]*0.5+1/k_
        S[i][ascending_order_index]=S[i][ascending_order_index]-dist_of_neighbours_sorted_ascending/alpha[i]*0.5
    mean_alpha=np.mean(alpha)
    S=np.maximum(S,0)
    
    result=0
    for k in range(v):
        a=0
        for i in range(n):
            for j in range(n):
                a+=S[i][j]*(la.norm(X[k][i]-X[k][j]))**2
        result+=a**0.5
    result+=mean_alpha*(la.norm(S))**2
    print('After initialization, the objective value is %f.'%result)
    
    #Update:
    for iter_num in range(30):
        #Update W_v:
        for k in range(v):
            tmp=0
            for i in range(n):
                for j in range(n):
                    tmp+=S[i][j]*((la.norm(X[k][i]-X[k][j]))**2)
            W[k]=0.5/(tmp**0.5)
        #Update F:
        diagonal_element=(np.sum(S,axis=1)+np.sum(S.T,axis=1))*0.5
        D_s=np.zeros((n,n),dtype=np.float32)#D_s denotes the degree matrix
        for i in range(n):
            D_s[i][i]=diagonal_element[i]
        L_s=D_s-(S+S.T)/2 #L_s denotes Laplacian matrix
        F[l:]=-np.dot(np.dot(la.inv(L_s[u:,l:]),L_s[u:,:l]),Yl)
        #Update S:
        S=np.zeros((n,n),dtype=np.float32)
        for i in range(n):
            for j in range(n):
                Dist[i][j]=0
                Dist[i][j]+=lambda_*la.norm(F[i,:]-F[j,:])**2 
                for k in range(v):
                    Dist[i][j]+=W[k]*(la.norm(X[k][i,:]-X[k][j,:]))**2
            ascending_order_index=np.argsort(Dist[i])[1:k_+1]
            dist_of_neighbours_sorted_ascending=Dist[i][ascending_order_index]
            alpha[i]=0.5*(k_*dist_of_neighbours_sorted_ascending[k_-1]-np.sum(dist_of_neighbours_sorted_ascending))
            S[i][ascending_order_index]=(np.mean(dist_of_neighbours_sorted_ascending))/alpha[i]*0.5+1/k_
            S[i][ascending_order_index]=S[i][ascending_order_index]-dist_of_neighbours_sorted_ascending/alpha[i]*0.5
        mean_alpha=np.mean(alpha)
        S=np.maximum(S,0)
        
        #Compute the objective value:
        result=0
        for k in range(v):
            a=0
            for i in range(n):
                for j in range(n):
                    a+=S[i][j]*(la.norm(X[k][i]-X[k][j]))**2
            result+=a**0.5
        result+=mean_alpha*(la.norm(S))**2
        print('After iteration %i,'%(iter_num+1),end='')
        print(' the objective value is %f.'%result)
        
    return F